In [1]:
from carbon.sdk import CarbonSDK, Tokens as T
#from carbon import CarbonOrderUI, CarbonSimulatorUI
from carbon.helpers.widgets import CheckboxManager, DropdownManager, PcSliderManager
from matplotlib import pyplot as plt
import numpy as np
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonSDK))
!node --version

CarbonSDK v0.9 (11/Mar/2023)
v18.14.2


# Taker Functionality [NB30]

_this Notebook gives an overview over the taker functionality; for more streamlined examples see the following notebooks_

In order to start the server, please run

    source /Volumes/Private/bin/sdkserver
    node server/sdkserver.mjs

In [2]:
SDK = CarbonSDK(disclaimer=False, verbose=True, Tokens=T)
pairs, pairs_s = SDK.pairs(inclstr=True)
SDK.version().get("msg")

[req0] method=post, url=http://localhost:3118/api/scall/pairs, params={}
[req0] method=get, url=http://localhost:3118/, params={}


'Carbon SDK v0.9 (7/Mar/2022)'

## Pair selection

In [3]:
try:
    pairs_sel = [s.strip() for s in pairs_s.split(",")]
    pairs_w()
except:
    pairs_w = DropdownManager(pairs_sel, defaultval="USDC/ETH")
    pairs_w()

Dropdown(options=(('USDC/ETH', 'USDC/ETH'),), value='USDC/ETH')

In [4]:
try:
    inverse_w()
except:
    inverse_w = CheckboxManager(["inverse"], values=[1])
    inverse_w()

In [5]:
tknb,tknq = pairs_w.value.split("/")
if inverse_w.values[0]:
    tknb,tknq = tknq, tknb
pair = f"{tknb}/{tknq}"
pair

'ETH/USDC'

## Market information

In [6]:
m = SDK.mGetMarketByPair(pair=pair)
m

[req0] method=post, url=http://localhost:3118/api/scall/get_min_rate_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}
[req0] method=post, url=http://localhost:3118/api/scall/get_max_rate_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}
[req0] method=post, url=http://localhost:3118/api/scall/get_min_rate_by_pair, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'}
[req0] method=post, url=http://localhost:3118/api/scall/get_max_rate_by_pair, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'}
[req0] method=post, url=http://localhost:3118/api/scall/get_liquidity_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0

MarketByPair(pair='ETH/USDC', bestBid=1599.9999998788553, bestAsk=2500.0, liqBid=2000.0, liqAsk=102.0, lastBid=1499.999999795006, lastAsk=3000.0000000000327)

## Trading examples

#### `mPrepareTrade`

In [7]:
approx_price = SDK.roundsd(m.mid,3)
amount_tknb = 1000
approx_price, amount_tknb

(2050.0, 1000)

In [8]:
r1 = SDK.mPrepareTrade(pair, SDK.SELL, tknb, amount_tknb/approx_price)
r1

[req0] method=post, url=http://localhost:3118/api/scall/get_trade_data, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'amount': '0.4878048780487805', 'tradeByTargetAmount': False, 'filter': []}


TradeData(pair='ETH/USDC', tkn='ETH', traderBuySell='SELL', amount=0.4878048780487805, sourceToken=SDKToken(token='ETH', address='0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', decimals=18, name='ETH'), targetToken=SDKToken(token='USDC', address='0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', decimals=6, name='USDC'), tradeByTargetAmount=False, tradeActions=[{'strategyId': 2, 'amount': 322748612218223762}, {'strategyId': 3, 'amount': 165056265830556738}], actionsWei=[{'id': 2, 'input': 322748612218223762, 'output': 500000000}, {'id': 3, 'input': 165056265830556738, 'output': 259733792}], actionsTokenRes=[{'id': 2, 'sourceAmount': 0.32274861221822376, 'targetAmount': 500.0}, {'id': 3, 'sourceAmount': 0.16505626583055674, 'targetAmount': 259.733792}], tradeData={'totalSourceAmount': 0.4878048780487805, 'totalTargetAmount': 758.594191, 'effectiveRate': 1555.11809155})

In [9]:
r2 = SDK.mPrepareTrade(pair, SDK.BUY, tknb, amount_tknb/approx_price)
r2

[req0] method=post, url=http://localhost:3118/api/scall/get_trade_data, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'amount': '0.4878048780487805', 'tradeByTargetAmount': True, 'filter': []}


TradeData(pair='ETH/USDC', tkn='ETH', traderBuySell='BUY', amount=0.4878048780487805, sourceToken=SDKToken(token='USDC', address='0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', decimals=6, name='USDC'), targetToken=SDKToken(token='ETH', address='0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', decimals=18, name='ETH'), tradeByTargetAmount=True, tradeActions=[{'strategyId': 2, 'amount': 487804878048780500}], actionsWei=[{'id': 2, 'input': 487804878048780500, 'output': 1231174920}], actionsTokenRes=[{'id': 2, 'sourceAmount': 1231.17492, 'targetAmount': 0.4878048780487805}], tradeData={'totalSourceAmount': 1233.024457, 'totalTargetAmount': 0.4878048780487805, 'effectiveRate': 0.0003956165470031553})

In [10]:
r3 = SDK.mPrepareTrade(pair, SDK.SELL, tknq, amount_tknb)
r3

[req0] method=post, url=http://localhost:3118/api/scall/get_trade_data, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'amount': '1000', 'tradeByTargetAmount': False, 'filter': []}


TradeData(pair='ETH/USDC', tkn='USDC', traderBuySell='SELL', amount=1000, sourceToken=SDKToken(token='USDC', address='0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', decimals=6, name='USDC'), targetToken=SDKToken(token='ETH', address='0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', decimals=18, name='ETH'), tradeByTargetAmount=False, tradeActions=[{'strategyId': 2, 'amount': 1000000000}], actionsWei=[{'id': 2, 'input': 1000000000, 'output': 396916857131052211}], actionsTokenRes=[{'id': 2, 'sourceAmount': 1000.0, 'targetAmount': 0.39691685713105224}], tradeData={'totalSourceAmount': 1000.0, 'totalTargetAmount': 0.39632148184535565, 'effectiveRate': 0.00039632148184535565})

In [11]:
r4 = SDK.mPrepareTrade(pair, SDK.BUY, tknq, amount_tknb)
r4

[req0] method=post, url=http://localhost:3118/api/scall/get_trade_data, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'amount': '1000', 'tradeByTargetAmount': True, 'filter': []}


TradeData(pair='ETH/USDC', tkn='USDC', traderBuySell='BUY', amount=1000, sourceToken=SDKToken(token='ETH', address='0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', decimals=18, name='ETH'), targetToken=SDKToken(token='USDC', address='0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', decimals=6, name='USDC'), tradeByTargetAmount=True, tradeActions=[{'strategyId': 2, 'amount': 500000000}, {'strategyId': 3, 'amount': 500000000}], actionsWei=[{'id': 2, 'input': 500000000, 'output': 322748612218223762}, {'id': 3, 'input': 500000000, 'output': 322748612218223762}], actionsTokenRes=[{'id': 2, 'sourceAmount': 0.32274861221822376, 'targetAmount': 500.0}, {'id': 3, 'sourceAmount': 0.32274861221822376, 'targetAmount': 500.0}], tradeData={'totalSourceAmount': 0.6464669248236831, 'totalTargetAmount': 1000.0, 'effectiveRate': 1546.8695483109818})

#### `mComposeTradeTransaction`

In [12]:
tx1 = SDK.mComposeTradeTransaction(**r1.cttkwargs)
tx1

[req0] method=post, url=http://localhost:3118/api/scall/compose_trade_by_source_transaction, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'tradeActions': [{'strategyId': '2', 'amount': '322748612218223762'}, {'strategyId': '3', 'amount': '165056265830556738'}], 'deadline': '4294967295', 'minReturn': '682.7347719', 'overrides': {'gasLimit': 999999999}}


{'data': '0xf1c5e014000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb4800000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff0000000000000000000000000000000000000000000000000000000028b1b4b300000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000047aa2271f047c920000000000000000000000000000000000000000000000000000000000000003000000000000000000000000000000000000000000000000024a65cc9cc86442',
 'to': '0x955a85437664fE809563dd0fA91B1590b159e1e6',
 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'},
 'value': {'type': 'BigNumber', 'hex': '0x06c507f3bbcce0d4'}}

In [13]:
tx2 = SDK.mComposeTradeTransaction(**r2.cttkwargs)
tx2

[req0] method=post, url=http://localhost:3118/api/scall/compose_trade_by_target_transaction, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'tradeActions': [{'strategyId': '2', 'amount': '487804878048780500'}], 'deadline': '4294967295', 'maxInput': '1356.3269027000001', 'overrides': {'gasLimit': 999999999}}


{'data': '0x102ee9ba000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff0000000000000000000000000000000000000000000000000000000050d7e7f60000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000006c507f3bbcce0d4',
 'to': '0x955a85437664fE809563dd0fA91B1590b159e1e6',
 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'}}

In [14]:
tx3 = SDK.mComposeTradeTransaction(**r3.cttkwargs)
tx3

[req0] method=post, url=http://localhost:3118/api/scall/compose_trade_by_source_transaction, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'tradeActions': [{'strategyId': '2', 'amount': '1000000000'}], 'deadline': '4294967295', 'minReturn': '0.3566893336608201', 'overrides': {'gasLimit': 999999999}}


{'data': '0xf1c5e014000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff00000000000000000000000000000000000000000000000004f3370f2406228400000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000003b9aca00',
 'to': '0x955a85437664fE809563dd0fA91B1590b159e1e6',
 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'}}

In [15]:
tx4 = SDK.mComposeTradeTransaction(**r4.cttkwargs)
tx4

[req0] method=post, url=http://localhost:3118/api/scall/compose_trade_by_target_transaction, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'tradeActions': [{'strategyId': '2', 'amount': '500000000'}, {'strategyId': '3', 'amount': '500000000'}], 'deadline': '4294967295', 'maxInput': '0.7111136173060514', 'overrides': {'gasLimit': 999999999}}


{'data': '0x102ee9ba000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb4800000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff00000000000000000000000000000000000000000000000009de62113a78634800000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000001dcd65000000000000000000000000000000000000000000000000000000000000000003000000000000000000000000000000000000000000000000000000001dcd6500',
 'to': '0x955a85437664fE809563dd0fA91B1590b159e1e6',
 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'},
 'value': {'type': 'BigNumber', 'hex': '0x3b9aca00'}}

#### Submit transactions

In [16]:
SDK.signsubmittx(tx1, sign=True)

[req0] method=post, url=http://localhost:3118/api/signsubmittx, params={'tx': {'data': '0xf1c5e014000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb4800000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff0000000000000000000000000000000000000000000000000000000028b1b4b300000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000047aa2271f047c920000000000000000000000000000000000000000000000000000000000000003000000000000000000000000000000000000000000000000024a65cc9cc86442', 'to': '0x955a85437664fE809563dd0fA91B1590b159e1e6', 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'}, 'value': {'type': 'BigNumber', 'hex': '0x06c507f3bbcce0d4'}}, True: True}


{'txReceipt': {'hash': '0xf800513ac49d6d999e101035d65823f3214bac25f11dc1508441c842241b0ce4',
  'nonce': None,
  'gasLimit': None,
  'gasPrice': None,
  'data': None,
  'value': None,
  'chainId': None,
  'confirmations': 0,
  'from': None}}

In [17]:
SDK.signsubmittx(tx2, sign=True)

[req0] method=post, url=http://localhost:3118/api/signsubmittx, params={'tx': {'data': '0x102ee9ba000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff0000000000000000000000000000000000000000000000000000000050d7e7f60000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000006c507f3bbcce0d4', 'to': '0x955a85437664fE809563dd0fA91B1590b159e1e6', 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'}}, True: True}


{'txReceipt': {'hash': '0x2ccad8ae88a20170eb7cc8d546fb462203cecc5bdb63e4fe70c64c3cb6e4cc36',
  'nonce': None,
  'gasLimit': None,
  'gasPrice': None,
  'data': None,
  'value': None,
  'chainId': None,
  'confirmations': 0,
  'from': None}}

In [18]:
SDK.signsubmittx(tx3, sign=True)

[req0] method=post, url=http://localhost:3118/api/signsubmittx, params={'tx': {'data': '0xf1c5e014000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff00000000000000000000000000000000000000000000000004f3370f2406228400000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000003b9aca00', 'to': '0x955a85437664fE809563dd0fA91B1590b159e1e6', 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'}}, True: True}


{'txReceipt': {'hash': '0x5bbb8e17e78bc610c21c38e22d433deef5ac7cab1f0a69ac40951eb456ae31b7',
  'nonce': None,
  'gasLimit': None,
  'gasPrice': None,
  'data': None,
  'value': None,
  'chainId': None,
  'confirmations': 0,
  'from': None}}

In [19]:
SDK.signsubmittx(tx4, sign=True)

[req0] method=post, url=http://localhost:3118/api/signsubmittx, params={'tx': {'data': '0x102ee9ba000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb4800000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff00000000000000000000000000000000000000000000000009de62113a78634800000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000001dcd65000000000000000000000000000000000000000000000000000000000000000003000000000000000000000000000000000000000000000000000000001dcd6500', 'to': '0x955a85437664fE809563dd0fA91B1590b159e1e6', 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'}, 'value': {'type': 'BigNumber', 'hex': '0x3b9aca00'}}, True: True}


{'txReceipt': {'hash': '0xa4d30f83b65ee4037106fe40e78d4136a883ba46619444137ad6c7810b341bb2',
  'nonce': None,
  'gasLimit': None,
  'gasPrice': None,
  'data': None,
  'value': None,
  'chainId': None,
  'confirmations': 0,
  'from': None}}